In [1]:
import ipywidgets as widgets
from SPARQLWrapper import SPARQLWrapper, JSON

class HealthDM(object):
    profile = None
    profilePresent = None
    profileAbsent = None
    
    sparqlQuery = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
PREFIX mesh2015: <http://id.nlm.nih.gov/mesh/2015/>
PREFIX mesh2016: <http://id.nlm.nih.gov/mesh/2016/>
PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>

SELECT ?d
FROM <http://id.nlm.nih.gov/mesh>
WHERE {{
    ?d a meshv:Term .
    {{?d meshv:prefLabel ?l
     FILTER (lcase(str(?l)) = lcase('{0}'))}}
    UNION
    {{?d meshv:altLabel ?l
     FILTER (lcase(str(?l)) = lcase('{0}'))}}
}}
ORDER BY ?d
"""
    
    # @classmethod
    # def setProfile(cls, profile):
    #     cls.profile = profile
    
    @classmethod
    def createProfile(cls):
        cls.profile = widgets.Accordion(children=[])
        return cls.profile
        
    @classmethod
    def clearTerms(cls):
        cls.profilePresent = widgets.Accordion(children=[])
        cls.profileAbsent = widgets.Accordion(children=[])
        cls.profile.children = tuple([cls.profilePresent, cls.profileAbsent])
        cls.profile.set_title(0, "Sign Present")
        cls.profile.set_title(1, "Sign Absent")

    @classmethod
    def addTerm(cls, heading, description, code, present, detail1, rate1, detail2, rate2):
        wprofile = [widgets.Text(description = "MeSH Term:", value = heading),
                    widgets.Text(description = "MeSH Code:", value = code)]
        if detail1 != "#":
            fDetail = cls.formatDetail(detail1, rate1)
            wprofile.append(widgets.Text(description = fDetail[0], value = fDetail[1]))
        if detail2 != "#":
            fDetail = cls.formatDetail(detail2, rate2)
            wprofile.append(widgets.Text(description = fDetail[0], value = fDetail[1]))
        wcontent = widgets.VBox(wprofile)
        profileList = cls.profilePresent if present else cls.profileAbsent
        profileList.children = tuple(list(profileList.children) + [wcontent])
        profileList.set_title(len(profileList.children)-1, description)
        
    @classmethod
    def formatDetail(cls, detail, rate):
        detailDesc = "Detail:"
        detailValue = detail
        if detail in ["occasionally", "often", "frequently", "sometimes",
                      "rarely", "daily", "monthly", "yearly", "each day",
                      "each month", "each year", "once a day", "once a month",
                      "once a year"]:
            detailDesc = "Frequency:"
        else:
            try:
                value = int(detail)
                detailDesc = "Intensity:"
                if rate != "#":
                    detailDesc = "Frequency:"
                    detailValue = detailValue + " / " + rate
            except ValueError:
                pass
        return [detailDesc, detailValue]

    @classmethod
    def findMeshCode(cls, heading, detail1, rate1, description, detail2, rate2):
        code = "none"
        present = True
        meshHeading = heading
        
        sparql = SPARQLWrapper("http://id.nlm.nih.gov/mesh/sparql")
        sparql.setReturnFormat(JSON)

        # looking for the heading
        sparql.setQuery(cls.sparqlQuery.format(meshHeading))
        results = sparql.query().convert()
        if len(results["results"]["bindings"]) > 0:
            code = results["results"]["bindings"][0]["d"]["value"]

        if code == "none" and (heading.lower().startswith("no") or
                               heading.lower().startswith("not")):
            # looking for the heading
            present = False
            meshHeading = heading[3:] if heading.lower().startswith("no") else heading[4:]
            sparql.setQuery(cls.sparqlQuery.format(meshHeading))
            results = sparql.query().convert()
            if len(results["results"]["bindings"]) > 0:
                code = results["results"]["bindings"][0]["d"]["value"]
            
        cls.addTerm(meshHeading, description, code, present, detail1, rate1, detail2, rate2)
    
        return "#mesh_heading#" + heading + "#tree_number#" + code
    
    @classmethod
    def interfacePresentation(cls, title, description, template):
        indexTemplate = open("template/index.html", "r", encoding="utf-8")
        presentationTemplate = open("template/Presentation.html", "r", encoding="utf-8")

        indexResult = open("html/index.html", "w", encoding="utf-8")
        presentationResult = open("html/Presentation.html", "w", encoding="utf-8")
        
        indexResult.write(indexTemplate.read().format(title=title))
        presentationResult.write(presentationTemplate.read().format(title=title, description=description))

        indexResult.close()
        presentationResult.close()

    @classmethod
    def interfaceKnot(cls, htmlName, title, description, template):
        knotTemplate = open("template/" + template + ".html", "r", encoding="utf-8")

        knotResult = open("html/" + htmlName + ".html", "w", encoding="utf-8")
        
        knotResult.write(knotTemplate.read().format(title=title, description=description))

        knotResult.close()

HealthDM.createProfile()

Accordion()

=== Zombie Health ===
Você é um Médico Girafa (girafas não viram zumbis).
=== Lingua ok (notice) ===
![Red_tongue](images/tongue-red.png)
A língua está vermelha.
=== Lingua problema (notice) ===
![Yellow_tongue](images/tongue-yellow.png)
A língua está amarela.
=== Paralisia ok (notice) ===
![No_paralysis](images/paralysis-no.png)
O está com bons movimentos.
=== Paralisia problema (notice) ===
![Paralysis](images/paralysis.png)
O corpo está paralisado.
=== Dedos ok (notice) ===
![Fingers_ok](images/fingers-ok.png)
A mãos não têm nenhuma irregularidade observável.
=== Dedos problema (notice) ===
![Fingers_shaking](images/fingers-shaking.png)
Os dedos mindinhos estão tremendo.
=== Membros ok (notice) ===
![Members_ok](images/members-ok.png)
Todos os membros estão no lugar.
=== Membros problema (notice) ===
![Members_loss](images/members-loss.png)
Foram perdidos alguns membros.
=== Dor ok (notice) ===
![Pain_no](images/pain-no.png)
Não tem nenhuma queixa de dor.
=== Dor problema (notice) ===
![Pain_chest](images/pain-chest.png)
Sente uma forte dor no peito.
=== Raiva ok (notice) ===
![Pain_no](images/anger-no.png)
Está de bom humor apesar de tudo.
=== Raiva problema (notice) ===
![Pain_chest](images/anger.png)
Sente uma grande raiva.
=== Bacterias certo (notice right) ===
Parabéns você tratou corretamente uma infecção por bactérias.
=== Bacterias errado (notice wrong) ===
Infelizmente você fez o tratamento incorreto, era uma infecção por bactérias.
=== Virus certo (notice right) ===
Parabéns você tratou corretamente uma infecção por vírus.
=== Virus errado (notice wrong) ===
Infelizmente você fez o tratamento incorreto, era uma infecção por vírus.
=== Zulumbriga certo (notice right) ===
Parabéns você tratou corretamente uma infecção por zulumbriga.
=== Zulumbriga errado (notice wrong) ===
Infelizmente você fez o tratamento incorreto, era uma infecção por zulumbriga.
=== Brigas certo (notice right) ===
Parabéns você tratou corretamente problemas que surgiram por uma briga.
=== Brigas errado (notice wrong) ===
Infelizmente você fez o tratamento incorreto, eram problemas que surgiram por uma briga.

=== Caso 1 (case) ===
![Boorthoo](images/1.png)
Boorthoo se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua ok
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros problema
++ Sente dor? -> Dor ok
++ Ver humor -> Raiva problema
++ Aplicar tratamento -> Tratamento 1
=== Tratamento 1 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Brigas errado 
++ Tratar Vírus -> Brigas errado 
++ Tratar Zulumbriga -> Brigas errado 
++ Tratar Brigas -> Brigas certo


=== Caso 2 (case) ===
![Tieppaeth](images/2.png)
Tieppaeth se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua ok
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros problema
++ Sente dor? -> Dor problema
++ Ver humor -> Raiva problema
++ Aplicar tratamento -> Tratamento 2
=== Tratamento 2 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Brigas errado 
++ Tratar Vírus -> Brigas errado 
++ Tratar Zulumbriga -> Brigas errado 
++ Tratar Brigas -> Brigas certo


=== Caso 3 (case) ===
![Aubuthu](images/3.png)
Aubuthu se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua ok
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros problema
++ Sente dor? -> Dor problema
++ Ver humor -> Raiva ok
++ Aplicar tratamento -> Tratamento 3
=== Tratamento 3 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Zulumbriga errado 
++ Tratar Vírus -> Zulumbriga errado 
++ Tratar Zulumbriga -> Zulumbriga certo 
++ Tratar Brigas -> Zulumbriga errado


=== Caso 4 (case) ===
![Umbratalon](images/4.png)
Umbratalon se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua problema
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros problema
++ Sente dor? -> Dor ok
++ Ver humor -> Raiva ok
++ Aplicar tratamento -> Tratamento 4
=== Tratamento 4 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Virus errado 
++ Tratar Vírus -> Virus certo 
++ Tratar Zulumbriga -> Virus errado 
++ Tratar Brigas -> Virus errado


=== Caso 5 (case) ===
![Berhtulf](images/5.png)
Berhtulf se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia problema
++ Examinar língua -> Lingua problema
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros ok
++ Sente dor? -> Dor ok
++ Ver humor -> Raiva ok
++ Aplicar tratamento -> Tratamento 5
=== Tratamento 5 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Virus errado 
++ Tratar Vírus -> Virus certo 
++ Tratar Zulumbriga -> Virus errado 
++ Tratar Brigas -> Virus errado


=== Caso 6 (case) ===
![Goknop](images/6.png)
Goknop se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua problema
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros problema
++ Sente dor? -> Dor ok
++ Ver humor -> Raiva ok
++ Aplicar tratamento -> Tratamento 6
=== Tratamento 6 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Virus errado 
++ Tratar Vírus -> Virus certo 
++ Tratar Zulumbriga -> Virus errado 
++ Tratar Brigas -> Virus errado


=== Caso 7 (case) ===
![Gimarwet](images/7.png)
Gimarwet se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua ok
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros problema
++ Sente dor? -> Dor problema
++ Ver humor -> Raiva problema
++ Aplicar tratamento -> Tratamento 7
=== Tratamento 7 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Brigas errado 
++ Tratar Vírus -> Brigas errado 
++ Tratar Zulumbriga -> Brigas errado 
++ Tratar Brigas -> Brigas certo


=== Caso 8 (case) ===
![Klinzbit](images/8.png)
Klinzbit se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua problema
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros ok
++ Sente dor? -> Dor ok
++ Ver humor -> Raiva ok
++ Aplicar tratamento -> Tratamento 8
=== Tratamento 8 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Virus errado 
++ Tratar Vírus -> Virus certo 
++ Tratar Zulumbriga -> Virus errado 
++ Tratar Brigas -> Virus errado


=== Caso 9 (case) ===
![Fnebnooss](images/9.png)
Fnebnooss se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua ok
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros problema
++ Sente dor? -> Dor problema
++ Ver humor -> Raiva ok
++ Aplicar tratamento -> Tratamento 9
=== Tratamento 9 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Zulumbriga errado 
++ Tratar Vírus -> Zulumbriga errado 
++ Tratar Zulumbriga -> Zulumbriga certo 
++ Tratar Brigas -> Zulumbriga errado


=== Caso 10 (case) ===
![Tasha Baba](images/10.png)
Tasha Baba se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua problema
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros problema
++ Sente dor? -> Dor ok
++ Ver humor -> Raiva ok
++ Aplicar tratamento -> Tratamento 10
=== Tratamento 10 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Virus errado 
++ Tratar Vírus -> Virus certo 
++ Tratar Zulumbriga -> Virus errado 
++ Tratar Brigas -> Virus errado


=== Caso 11 (case) ===
![Cifonglas](images/11.png)
Cifonglas se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua problema
++ Analisar as mãos -> Dedos problema
++ Examinar membros -> Membros problema
++ Sente dor? -> Dor ok
++ Ver humor -> Raiva ok
++ Aplicar tratamento -> Tratamento 11
=== Tratamento 11 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Bacterias certo 
++ Tratar Vírus -> Bacterias errado 
++ Tratar Zulumbriga -> Bacterias errado 
++ Tratar Brigas -> Bacterias errado


=== Caso 12 (case) ===
![Glinivum](images/12.png)
Glinivum se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia problema
++ Examinar língua -> Lingua problema
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros ok
++ Sente dor? -> Dor ok
++ Ver humor -> Raiva ok
++ Aplicar tratamento -> Tratamento 12
=== Tratamento 12 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Virus errado 
++ Tratar Vírus -> Virus certo 
++ Tratar Zulumbriga -> Virus errado 
++ Tratar Brigas -> Virus errado


=== Caso 13 (case) ===
![Zevansnoc](images/13.png)
Zevansnoc se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua ok
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros ok
++ Sente dor? -> Dor problema
++ Ver humor -> Raiva problema
++ Aplicar tratamento -> Tratamento 13
=== Tratamento 13 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Brigas errado 
++ Tratar Vírus -> Brigas errado 
++ Tratar Zulumbriga -> Brigas errado 
++ Tratar Brigas -> Brigas certo


=== Caso 14 (case) ===
![Batnisb](images/14.png)
Batnisb se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua ok
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros problema
++ Sente dor? -> Dor ok
++ Ver humor -> Raiva problema
++ Aplicar tratamento -> Tratamento 14
=== Tratamento 14 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Brigas errado 
++ Tratar Vírus -> Brigas errado 
++ Tratar Zulumbriga -> Brigas errado 
++ Tratar Brigas -> Brigas certo


=== Caso 15 (case) ===
![Rugdumph](images/15.png)
Rugdumph se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua problema
++ Analisar as mãos -> Dedos problema
++ Examinar membros -> Membros problema
++ Sente dor? -> Dor ok
++ Ver humor -> Raiva ok
++ Aplicar tratamento -> Tratamento 15
=== Tratamento 15 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Bacterias certo 
++ Tratar Vírus -> Bacterias errado 
++ Tratar Zulumbriga -> Bacterias errado 
++ Tratar Brigas -> Bacterias errado


=== Caso 16 (case) ===
![Sugigoth](images/16.png)
Sugigoth se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua problema
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros problema
++ Sente dor? -> Dor ok
++ Ver humor -> Raiva ok
++ Aplicar tratamento -> Tratamento 16
=== Tratamento 16 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Virus errado 
++ Tratar Vírus -> Virus certo 
++ Tratar Zulumbriga -> Virus errado 
++ Tratar Brigas -> Virus errado


=== Caso 17 (case) ===
![Wumkbanok](images/17.png)
Wumkbanok se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua ok
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros problema
++ Sente dor? -> Dor problema
++ Ver humor -> Raiva problema
++ Aplicar tratamento -> Tratamento 17
=== Tratamento 17 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Brigas errado 
++ Tratar Vírus -> Brigas errado 
++ Tratar Zulumbriga -> Brigas errado 
++ Tratar Brigas -> Brigas certo


=== Caso 18 (case) ===
![Elvobarin](images/18.png)
Elvobarin se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua ok
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros problema
++ Sente dor? -> Dor problema
++ Ver humor -> Raiva ok
++ Aplicar tratamento -> Tratamento 18
=== Tratamento 18 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Zulumbriga errado 
++ Tratar Vírus -> Zulumbriga errado 
++ Tratar Zulumbriga -> Zulumbriga certo 
++ Tratar Brigas -> Zulumbriga errado


=== Caso 19 (case) ===
![Vokalis](images/19.png)
Vokalis se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua ok
++ Analisar as mãos -> Dedos ok
++ Examinar membros -> Membros ok
++ Sente dor? -> Dor problema
++ Ver humor -> Raiva ok
++ Aplicar tratamento -> Tratamento 19
=== Tratamento 19 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Zulumbriga errado 
++ Tratar Vírus -> Zulumbriga errado 
++ Tratar Zulumbriga -> Zulumbriga certo 
++ Tratar Brigas -> Zulumbriga errado


=== Caso 20 (case) ===
![Tolith Traphyra](images/20.png)
Tolith Traphyra se apresentou na clínica para ser atendido. O que você deseja fazer?
++ Ver movimentos -> Paralisia ok
++ Examinar língua -> Lingua ok
++ Analisar as mãos -> Dedos problema
++ Examinar membros -> Membros ok
++ Sente dor? -> Dor ok
++ Ver humor -> Raiva ok
++ Aplicar tratamento -> Tratamento 20
=== Tratamento 20 ===
![Treatment](images/doctor.png)
Qual é o tratamento que você deseja aplicar?
++ Tratar Bactérias -> Bacterias certo 
++ Tratar Vírus -> Bacterias errado 
++ Tratar Zulumbriga -> Bacterias errado 
++ Tratar Brigas -> Bacterias errado

In [2]:
%%HTML
<iframe width="100%" height="800px" src="html/index.html"><iframe>